In [42]:
import os
import csv
from dotenv import load_dotenv
import time
import requests
import json
import random

In [ ]:
load_dotenv()

XAI_API_KEY = os.getenv("XAI_API_KEY") 

In [ ]:
API_URL = "https://api.x.ai/v1/chat/completions"
MODEL = "grok-2-vision-1212"

HEADERS = {
    "Authorization": f"Bearer {XAI_API_KEY}",
    "Content-Type": "application/json"
}

In [45]:
def read_questions(csv_path):
    questions = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, question = row[0].strip(), row[1].strip()
                questions[num] = question
    return questions

In [46]:
def read_options(csv_path):
    options = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, answer = row[0].strip(), row[1].strip()
                options.setdefault(num, []).append(answer)
    return options

In [ ]:
def read_descriptions(csv_path):
    descriptions = {}
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                num, desc = row[0].strip(), row[1].strip()
                descriptions[num] = desc
    return descriptions

In [ ]:
def ask_grok_about_description(description, question, options_list):

    system_prompt = (
        "You are a specialized Latvian traffic rules expert. "
        "Your entire reasoning and final output MUST be in English.\n"
        "ABSOLUTE OUTPUT REQUIREMENT:\n"
        "- Output ONLY the exact text of the single correct option (in English).\n"
        "- Do NOT include any reasoning, explanations, numbering, quotes, or extra text.\n"
        "- The output must match one of the provided options verbatim."
    )

    user_prompt = (
        f"Description of the image:\n{description}\n\n"
        f"Question:\n{question}\n\n"
        f"Options:\n" + "\n".join(options_list) if options_list else "No options provided"
    )

    payload = {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_prompt}
        ],
        "model": MODEL,
        "temperature": 0.0, # More deterministic answer
        "max_tokens": 500,
        "top_p": 1.0
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload, timeout=60)
        response.raise_for_status()
        data = response.json()

        answer = data["choices"][0]["message"]["content"].strip()

        answer = answer.strip('"\' \n')
        return answer

    except requests.exceptions.HTTPError as http_err:
        err_detail = response.text
        return f"HTTP Error: {http_err} – {err_detail}"
    except Exception as e:
        return f"API Error: {str(e)}"

In [ ]:
def main():
    descriptions_csv = "descriptions.csv"
    questions_csv    = "questions.csv"
    options_csv      = "options.csv"

    descriptions = read_descriptions(descriptions_csv)
    questions    = read_questions(questions_csv)
    options      = read_options(options_csv)

    all_nums = sorted(set(descriptions.keys()) & set(questions.keys()))

    random.shuffle(all_nums)

    print(f"Processing {len(all_nums)} descriptions in random order.")

    results_all = []

    count = 1
    for q_num in all_nums:
        description = descriptions[q_num]
        question    = questions[q_num]
        answer_opts = options.get(q_num, [])

        print(f"Processing description {q_num}... ({count}/{len(all_nums)})")

        answer = ask_grok_about_description(description, question, answer_opts)

        count += 1

        results_all.append({
            "image_number": q_num,
            "model_answer": answer
        })
        print(f"→ {answer}")

    output_csv_file = "results_Grok_description.csv"
    with open(output_csv_file, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(
            f, fieldnames=["image_number", "question", "options", "model_answer"]
        )
        writer.writeheader()
        for res in results_all:
            q_num = res["image_number"]
            writer.writerow({
                "image_number": q_num,
                "question": questions.get(q_num, ""),
                "options": "; ".join(options.get(q_num, [])),
                "model_answer": res["model_answer"]
            })

    print(f"\nAll done! Results saved to: {output_csv_file}")

In [50]:
if __name__ == "__main__":
    main()

Processing 100 descriptions in random order.
Processing description 79... (1/100)
→ Cycleway will cross the carriageway.
Processing description 10... (2/100)
→ Only straight and to the left.
Processing description 17... (3/100)
→ B D
Processing description 47... (4/100)
→ It is forbidden.
Processing description 89... (5/100)
→ It is forbidden.
Processing description 62... (6/100)
→ To all.
Processing description 82... (7/100)
→ No.
Processing description 15... (8/100)
→ Prohibited.
Processing description 29... (9/100)
→ You will give way to the car driver.
Processing description 61... (10/100)
→ Allowed, if the cargo does not interfere with driving and does not disturb other traffic participants
Processing description 48... (11/100)
→ On a priority road.
Processing description 38... (12/100)
→ Prohibited where the roadway is crossed by a bicycle path.
Processing description 7... (13/100)
→ You will give way to the cyclist.
Processing description 56... (14/100)
→ I will cross last.
Proc